In [1]:
# -*- coding: UTF-8 -*-
import time
import pandas as pd
import pickle
import os
from datetime import datetime,timedelta
from sklearn.ensemble import GradientBoostingClassifier
import random
import xgboost as xgb
import numpy as np
from sklearn.model_selection import train_test_split
from operator import itemgetter
import operator
from sklearn.model_selection import KFold
import scipy as sp
import gc
import lightgbm as lgb
from collections import Counter
from collections import defaultdict

In [2]:
app_cat = pd.read_csv('data/app200_category.csv')
tot_actions = pd.read_csv('total_action.csv')

In [37]:
print tot_actions.columns
print app_cat.columns

Index([u'userid', u'app_id', u'action_type', u'action_time'], dtype='object')
Index([u'app_id', u'des', u'cat1', u'cat2'], dtype='object')


In [5]:
#app分类map
app_cat1 = []
for i in app_cat['cat1']:
    if (i == 0):
        app_cat1.append(4)
    else:
        app_cat1.append(i)
del app_cat['cat1']
app_cat['cat1'] = app_cat1

app_map = defaultdict(int)
for i, dx in app_cat.iterrows():
    app_map[dx.app_id] = dx.cat1

In [69]:
#计算同一类下app的安装，卸载，下载个数
import csv
nd = {}
num = 0

with open('total_action.csv','rb') as csvfile:
    tot_actions = csv.DictReader(csvfile)

    for rec in tot_actions:
        
        if (int(rec['action_time']) > 272359):
            continue

        num += 1

        actype = 0
        if (rec['action_type'] == 'DOWNLOADED'):
            actype = 1
        elif(rec['action_type'] == 'INSTALLED'):
            actype = 2

        key = str(rec['userid']) + ',' + str(app_map[rec['app_id']])

        if (not nd.has_key(key)):
            nd[key] = [0, 0, 0]

        nd[key][actype] += 1

        if (num %10000000 == 0):
            print num
            
        

10000000
20000000
30000000
40000000
50000000
60000000


In [2]:
d28 = pd.read_csv('280000.csv')
d29 = pd.read_csv('290000.csv')
d30 = pd.read_csv('300000.csv')

traindata = pd.concat([d28, d29])
testdata = d30.copy()

print len(d28) + len(d29)
print len(traindata)

1806748
1806748


In [70]:
#traindata，testdata拼接actiontype
testact0 = []
testact1 = []
testact2 = []

with open('300000.csv','rb') as csvfile:
    actions30 = csv.DictReader(csvfile)

    for rec in actions30:
        testact0.append(nd.get(str(rec['user_id']) + ',' + str(app_map[rec['app_id']]), [-1, -1, -1])[0])
        testact1.append(nd.get(str(rec['user_id']) + ',' + str(app_map[rec['app_id']]), [-1, -1, -1])[1])
        testact2.append(nd.get(str(rec['user_id']) + ',' + str(app_map[rec['app_id']]), [-1, -1, -1])[2])

print 'test done'

trainact0 = []
trainact1 = []
trainact2 = []

for fn in ['280000.csv', '290000.csv']:
    with open(fn, 'rb') as csvfile:
        actions33 = csv.DictReader(csvfile)

        for rec in actions33:
            trainact0.append(nd.get(str(rec['user_id']) + ',' + str(app_map[rec['app_id']]), [-1, -1, -1])[0])
            trainact1.append(nd.get(str(rec['user_id']) + ',' + str(app_map[rec['app_id']]), [-1, -1, -1])[1])
            trainact2.append(nd.get(str(rec['user_id']) + ',' + str(app_map[rec['app_id']]), [-1, -1, -1])[2])
            
print 'train done'

test done
train done


In [73]:
del traindata['action0']
del traindata['action1']
del traindata['action2']

del testdata['action0']
del testdata['action1']
del testdata['action2']

In [74]:
traindata['action0'] = trainact0
traindata['action1'] = trainact1
traindata['action2'] = trainact2

testdata['action0'] = testact0
testdata['action1'] = testact1
testdata['action2'] = testact2

In [ ]:
shopping = pd.read_csv('./data/shopping_data.csv')

In [ ]:
print shopping

In [19]:
def fun(x):
    sl = x[0].split(',')
    return len(sl)

shopping['shopping_num'] = shopping[['shopping']].apply(fun, axis = 1)

In [21]:
user_shopping = shopping[['user_id', 'shopping_num']].copy()

In [27]:
from collections import defaultdict  
dd = defaultdict(int)

for i, dx in user_shopping.iterrows():
    dd[dx.user_id] += dx.shopping_num


In [29]:
trainmap = []
testmap = []

for i in traindata['user_id']:
    trainmap.append(dd[i])
for i in testdata['user_id']:
    testmap.append(dd[i])

traindata['user_shopping'] = trainmap
testdata['user_shopping'] = testmap

In [78]:
tot_usage = pd.read_csv('total_usage.csv')
print len(tot_usage)

96329381


In [3]:
shopping = pd.read_csv('shopping_feature.csv')

In [4]:
print shopping.columns

Index([u'instance_id', u'label', u'sum_t'], dtype='object')


In [8]:
del shopping['label']

In [12]:
print len(traindata)
traindata = pd.merge(traindata, shopping, how = 'left', on = 'instance_id')
print len(traindata)
print len(testdata)
testdata = pd.merge(testdata, shopping, how = 'left', on = 'instance_id')
print len(testdata)

1806748
1806748
893540
893540


In [13]:
print traindata.columns

Index([u'instance_id', u'label', u'click_time', u'ad_id', u'user_id',
       u'position_id', u'connection_type', u'MIUI_version', u'IP',
       u'android_version', u'advertiser_id', u'compaign_id', u'app_id', u'age',
       u'gender', u'education', u'provice', u'city', u'device_info',
       u'app_category1', u'app_category2', u'ad_id_click', u'ad_id_use',
       u'ad_id_click_maen', u'ad_id_use_maen', u'ad_id_ratio',
       u'compaign_id_click', u'compaign_id_use', u'compaign_id_click_maen',
       u'compaign_id_use_maen', u'compaign_id_ratio', u'advertiser_id_click',
       u'advertiser_id_use', u'advertiser_id_click_maen',
       u'advertiser_id_use_maen', u'advertiser_id_ratio', u'app_id_click',
       u'app_id_use', u'app_id_click_maen', u'app_id_use_maen',
       u'app_id_ratio', u'user_install_app_pre', u'user_usage_app_count',
       u'user_usage_app_time', u'user_usage_app_count_mean',
       u'user_usage_app_time_mean', u'user_id_app_id_count',
       u'user_id_app_id_use', u

In [15]:
traindata = traindata.fillna(-1)
testdata = testdata.fillna(-1)
traindata = traindata.replace({np.inf: -1})
testdata = testdata.replace({np.inf: -1})

predictors = [
       'ad_id', 'user_id',
       'position_id', 'connection_type', 'MIUI_version', 'IP',
       'android_version', 'advertiser_id', 'compaign_id', 'app_id', 'age',
       'gender', 'education', 'provice', 'city', 'device_info',
       'app_category1', 'app_category2', 'ad_id_click', 'ad_id_use',
       'ad_id_click_maen', 'ad_id_use_maen', 'ad_id_ratio',
       'compaign_id_click', 'compaign_id_use', 'compaign_id_click_maen',
       'compaign_id_use_maen', 'compaign_id_ratio', 'advertiser_id_click',
       'advertiser_id_use', 'advertiser_id_click_maen',
       'advertiser_id_use_maen', 'advertiser_id_ratio', 'app_id_click',
       'app_id_use', 'app_id_click_maen', 'app_id_use_maen',
       'app_id_ratio', 'user_install_app_pre', 'user_usage_app_count',
       'user_usage_app_time', 'user_usage_app_count_mean',
       'user_usage_app_time_mean', 'user_id_app_id_count',
       'user_id_app_id_use', 'user_app_idratio', 'user_id_ad_id_count',
       'user_id_ad_id_use', 'user_ad_idratio', 'sum_t'
    #'action0', 'action1', 'action2'
    #'down_cat', 'install_cat', 'uninstall_cat'
]

print(len(predictors))
print(len(traindata), len(testdata))

train1 = xgb.DMatrix(traindata[predictors], label=traindata['label'])
test1 = xgb.DMatrix(testdata[predictors], label=testdata['label'])

# params = {
#     'objective': 'binary:logistic',
#     'eta': 0.05,
#     'colsample_bytree': 0.886,
#     'min_child_weight': 2,
#     'max_depth': 10,
#     'subsample': 0.886,
#     'alpha': 10,
#     'gamma': 30,
#     'lambda':50,
#     'verbose_eval': True,
#    # 'nthread': 8,
#     'eval_metric': 'logloss',
#     'scale_pos_weight': 10,
#     'seed': 201703,
#     'missing':-1
# }

param = {
    'objective': 'binary:logistic',
    'eta': 0.05,
    'colsample_bytree': 0.8,
    'subsample': 0.8,
    'silent': 1,
    'verbose_eval': True,
    'eval_metric': 'logloss',
    'seed': 201703,
    'missing': -1
}

num_round = 300
plst = list(param.items())
evallist = [(train1, 'train'), (test1, 'eval')]

bst = xgb.train(plst, train1, num_round, evallist)

importance = bst.get_fscore()
importance = sorted(importance.items(), key=operator.itemgetter(1))
print(importance)
y = bst.predict(test1)
testdata['pred'] = y

50
(1806748, 893540)


[0]	train-logloss:0.668226	eval-logloss:0.668329
[1]	train-logloss:0.645237	eval-logloss:0.645466
[2]	train-logloss:0.624489	eval-logloss:0.624903
[3]	train-logloss:0.605299	eval-logloss:0.605859
[4]	train-logloss:0.588166	eval-logloss:0.588874
[5]	train-logloss:0.572171	eval-logloss:0.573043
[6]	train-logloss:0.557550	eval-logloss:0.558572
[7]	train-logloss:0.544183	eval-logloss:0.545308
[8]	train-logloss:0.531915	eval-logloss:0.533186
[9]	train-logloss:0.520625	eval-logloss:0.521983
[10]	train-logloss:0.510281	eval-logloss:0.511726
[11]	train-logloss:0.500751	eval-logloss:0.502313
[12]	train-logloss:0.492005	eval-logloss:0.493672
[13]	train-logloss:0.484091	eval-logloss:0.485816
[14]	train-logloss:0.476638	eval-logloss:0.478456
[15]	train-logloss:0.469776	eval-logloss:0.471676
[16]	train-logloss:0.463420	eval-logloss:0.465405
[17]	train-logloss:0.457570	eval-logloss:0.459622
[18]	train-logloss:0.452175	eval-logloss:0.454320
[19]	train-logloss:0.447192	eval-logloss:0.449395
[20]	train

[163]	train-logloss:0.381708	eval-logloss:0.387171
[164]	train-logloss:0.381688	eval-logloss:0.387172
[165]	train-logloss:0.381664	eval-logloss:0.387154
[166]	train-logloss:0.381648	eval-logloss:0.387148
[167]	train-logloss:0.381633	eval-logloss:0.387144
[168]	train-logloss:0.381617	eval-logloss:0.387136
[169]	train-logloss:0.381605	eval-logloss:0.387131
[170]	train-logloss:0.381575	eval-logloss:0.387113
[171]	train-logloss:0.381546	eval-logloss:0.387096
[172]	train-logloss:0.381531	eval-logloss:0.387084
[173]	train-logloss:0.381511	eval-logloss:0.387075
[174]	train-logloss:0.381474	eval-logloss:0.387043
[175]	train-logloss:0.381452	eval-logloss:0.387033
[176]	train-logloss:0.381431	eval-logloss:0.387022
[177]	train-logloss:0.381389	eval-logloss:0.387015
[178]	train-logloss:0.381368	eval-logloss:0.387003
[179]	train-logloss:0.381352	eval-logloss:0.386998
[180]	train-logloss:0.381334	eval-logloss:0.386990
[181]	train-logloss:0.381316	eval-logloss:0.386981
[182]	train-logloss:0.381287	ev

[('user_id_ad_id_use', 13), ('user_id_ad_id_count', 32), ('app_id_use_maen', 47), ('user_ad_idratio', 57), ('advertiser_id_use_maen', 57), ('user_id_app_id_use', 67), ('app_id_click_maen', 69), ('advertiser_id_click_maen', 81), ('compaign_id_use_maen', 98), ('compaign_id_click_maen', 108), ('app_id_use', 131), ('ad_id_use_maen', 148), ('gender', 150), ('app_category1', 154), ('advertiser_id_use', 155), ('app_id_click', 158), ('user_id_app_id_count', 176), ('compaign_id_use', 188), ('position_id', 189), ('education', 207), ('ad_id_click_maen', 214), ('user_usage_app_count_mean', 216), ('advertiser_id_click', 222), ('user_usage_app_count', 252), ('sum_t', 260), ('ad_id_use', 271), ('app_category2', 281), ('advertiser_id', 306), ('compaign_id_click', 307), ('advertiser_id_ratio', 316), ('user_app_idratio', 316), ('app_id', 320), ('connection_type', 323), ('user_usage_app_time_mean', 326), ('compaign_id', 330), ('user_install_app_pre', 365), ('app_id_ratio', 369), ('compaign_id_ratio', 372

[299]	train-logloss:0.379579	eval-logloss:0.386655
